# Technical Report: Socioeconomic Determinants of Hospital Readmissions and Mortality

### Names of the Team Members:
* Member 1: Haichao Min
* Member 2: Xuhan Wang

### Project Overview:

This project investigates a critical link: the relationship between hospital quality and the socioeconomic conditions of the communities they serve. Drawing on hospital performance data from the Centers for Medicare & Medicaid Services (CMS) and socioeconomic indicators from the U.S. Census Bureau’s American Community Survey (ACS), the analysis seeks to determine whether hospitals located in more disadvantaged areas experience systematically higher readmission and mortality rates. This research is particularly significant as it confronts structural inequities in healthcare delivery and access. Given the current potential budget cuts and their broader social impact, this study is especially timely, emphasizing how financial constraints and socioeconomic disparities may interact to influence healthcare quality and outcomes. Furthermore, it critically examines whether federal policies, such as the Hospital Readmissions Reduction Program, unintentionally penalize hospitals that serve the nation’s most vulnerable populations.

### Stakeholders:
- **Policymakers and Regulatory Bodies (e.g., CMS):** To inform adjustments to quality payment programs and health equity policies.

- **Hospital Administrators and Quality Improvement Officers:** To understand the external factors impacting performance metrics and to develop targeted community health initiatives.

- **Public Health Organizations and Researchers:** To provide evidence for health disparities and guide further research.

- **Patients and Community Advocates:** To raise awareness of potential inequities in healthcare access and quality.


### Objective:

To quantify the statistical relationship between county-level socioeconomic indicators (poverty, insurance coverage, income) and hospital quality outcomes (readmission and mortality rates), while controlling for hospital-specific characteristics.

### Detailed Project Description:

This project will conduct a cross-sectional analysis of U.S. hospitals to examine the association between community-level socioeconomic disadvantage and hospital performance on key clinical outcomes. We will merge hospital-level data from CMS with county-level socioeconomic data from the U.S. Census to address our central research questions.

**Business Questions:**
1. Are hospitals located in counties with higher poverty rates more likely to have higher 30-day readmission and mortality rates?
2. Does the rate of health insurance coverage at the county level correlate with hospital quality outcomes?
3. After adjusting for intrinsic hospital characteristics (such as type and ownership), do county-level socioeconomic conditions remain significant predictors of hospital performance?

### Literature Review and Motivation:

### Methodology:

Our methodology consists of four main stages: Data Sourcing and Integration, Data Cleaning and Preprocessing, Exploratory Data Analysis (EDA), and Statistical Modeling.


#### Data Overview & Sourcing:

We will use six publicly available datasets:

**CMS Datasets (Provider Data Catalog):**
    1.  **Hospital General Information:** Provides characteristics for each hospital, including Facility ID, name, location (county and state), hospital type, and ownership structure.
    2.  **Readmissions Reduction:** Contains hospital-level data on 30-day readmission measures, including the crucial `Excess Readmission Ratio`.
    3.  **Complications & Deaths:** Includes data on 30-day mortality rates for various conditions (e.g., heart attack, pneumonia), which we will use as our mortality outcome measure.

**U.S. Census ACS 2023 1-Year Estimates:**
    1.  **S1701 (Poverty Status):** Provides county-level estimates of the percentage of the population below the poverty line.
    2.  **S2701 (Health Insurance Coverage):** Contains county-level estimates for the percentage of the population without health insurance.
    3.  **DP03 (Selected Economic Characteristics):** Includes county-level data on median household income and labor force statistics.

#### Data Cleaning & Preprocessing:

The raw datasets require significant cleaning and merging to create a unified analytical file. The process is outlined below, with corresponding code snippets.

**Step 1: Load and Inspect Data**
First, we load all six CSV files into pandas DataFrames and perform an initial inspection.

In [6]:
import pandas as pd
import numpy as np

# Load CMS Datasets
hospital_info = pd.read_csv("Databases/Hospital_General_Information.csv")
readmissions = pd.read_csv("Databases/Readmissions_Reduction.csv")
complications = pd.read_csv("Databases/Complications & Deaths.csv")

# Load ACS Datasets
acs_income = pd.read_csv("Databases/ACS.DP03/ACSDP1Y2023.DP03-Data.csv")
acs_poverty = pd.read_csv("Databases/ACS.S1701/ACSST1Y2023.S1701-Data.csv")
acs_insurance = pd.read_csv("Databases/ACS.S2701/ACSST1Y2023.S2701-Data.csv")

print("Hospital Info Shape:", hospital_info.shape)
print("Readmissions Shape:", readmissions.shape)
print("Complications Shape:", complications.shape)
print("ACS Income Shape:", acs_income.shape)
print("ACS Poverty Shape:", acs_poverty.shape)
print("ACS Insurance Shape:", acs_insurance.shape)

Hospital Info Shape: (5381, 38)
Readmissions Shape: (18510, 12)
Complications Shape: (95100, 18)
ACS Income Shape: (855, 551)
ACS Poverty Shape: (855, 375)
ACS Insurance Shape: (855, 613)


**Step 2: Clean and Merge CMS Data**

 The CMS datasets are at the hospital or hospital-measure level. We will aggregate them to create a single file with one row per hospital, containing key characteristics and averaged outcome measures.

- **Filter Measures:** We will focus on hospital-wide readmission (`READM-30-HOSP-WIDE-HRRP`) and mortality (`MORT_30_...`) measures.

- **Handle Missing Values:** Columns like `Score` and `Excess Readmission Ratio` contain non-numeric values (e.g., 'Not Available'). These will be converted to numeric types, coercing errors into `NaN` (Not a Number), which can then be handled or dropped.

- **Aggregate Data:** We will calculate the average mortality rate for each hospital across different conditions and select the hospital-wide readmission ratio.
    
- **Merge:** The cleaned `readmissions` and `complications` data will be merged with the `hospital_info` DataFrame using `Facility ID` as the key.

In [12]:
# --- Clean Hospital Info Data ---
hospital_info_cleaned = hospital_info[['Facility ID', 'Facility Name', 'County/Parish', 'State', 'Hospital Type', 'Hospital Ownership']].copy()
hospital_info_cleaned.rename(columns={'County/Parish': 'County'}, inplace=True)
# Ensure Facility ID is string type for consistent merging
hospital_info_cleaned['Facility ID'] = hospital_info_cleaned['Facility ID'].astype(str)

# --- Clean Readmissions Data (Robust Method) ---
# Instead of filtering for one specific measure, we average the ratio across ALL available readmission measures for each hospital.
readmissions['Excess Readmission Ratio'] = pd.to_numeric(readmissions['Excess Readmission Ratio'], errors='coerce')
# Convert Facility ID to string to match hospital_info
readmissions['Facility ID'] = readmissions['Facility ID'].astype(str)
# Group by hospital and calculate the mean of their readmission ratios
readmissions_agg = readmissions.groupby('Facility ID')['Excess Readmission Ratio'].mean().reset_index()
readmissions_final = readmissions_agg.rename(columns={'Excess Readmission Ratio': 'Average Excess Readmission Ratio'})

# --- Clean Complications (Mortality) Data ---
mortality_measures = ['MORT_30_AMI', 'MORT_30_CABG', 'MORT_30_COPD', 'MORT_30_HF', 'MORT_30_PN', 'MORT_30_STK']
complications_cleaned = complications[complications['Measure ID'].isin(mortality_measures)].copy()
complications_cleaned['Score'] = pd.to_numeric(complications_cleaned['Score'], errors='coerce')
# Convert Facility ID to string to match hospital_info
complications_cleaned['Facility ID'] = complications_cleaned['Facility ID'].astype(str)
# Aggregate by taking the mean mortality score for each hospital
mortality_agg = complications_cleaned.groupby('Facility ID')['Score'].mean().reset_index()
mortality_agg.rename(columns={'Score': 'Average Mortality Rate'}, inplace=True)

# --- Merge CMS Datasets using a robust LEFT join strategy ---
# Start with the main hospital list
cms_merged = pd.merge(hospital_info_cleaned, readmissions_final, on='Facility ID', how='left')
# Add the mortality data
cms_merged = pd.merge(cms_merged, mortality_agg, on='Facility ID', how='left')

print("Merged CMS Data Shape (before dropping NaNs):", cms_merged.shape)
print("Hospitals with Readmission data:", cms_merged['Average Excess Readmission Ratio'].notna().sum())
print("Hospitals with Mortality data:", cms_merged['Average Mortality Rate'].notna().sum())

# For the analysis, we need hospitals with BOTH outcome measures.
analytical_set_count = cms_merged.dropna(subset=['Average Excess Readmission Ratio', 'Average Mortality Rate']).shape[0]
print(f"\nNumber of hospitals with BOTH readmission and mortality data: {analytical_set_count}")

cms_merged.dropna(subset=['Average Excess Readmission Ratio', 'Average Mortality Rate'], inplace=True)

print(f"\nFinal Merged CMS Data Shape (analytical set): {cms_merged.shape}")
cms_merged.head()

Merged CMS Data Shape (before dropping NaNs): (5381, 8)
Hospitals with Readmission data: 2335
Hospitals with Mortality data: 3696

Number of hospitals with BOTH readmission and mortality data: 2241

Final Merged CMS Data Shape (analytical set): (2241, 8)


,Facility ID,Facility Name,County,State,Hospital Type,Hospital Ownership,Average Excess Readmission Ratio,Average Mortality Rate
851,100001,SHANDS JACKSONVILLE,DUVAL,FL,Acute Care Hospitals,Voluntary non-profit - Private,1.045220,9.733333
852,100002,BETHESDA HOSPITAL EAST,PALM BEACH,FL,Acute Care Hospitals,Voluntary non-profit - Private,1.018333,11.600000
853,100006,ORLANDO HEALTH,ORANGE,FL,Acute Care Hospitals,Voluntary non-profit - Private,1.073800,9.183333
854,100007,ADVENTHEALTH ORLANDO,ORANGE,FL,Acute Care Hospitals,Voluntary non-profit - Private,1.165183,7.983333
855,100008,BAPTIST HOSPITAL OF MIAMI,MIAMI-DADE,FL,Acute Care Hospitals,Voluntary non-profit - Private,0.994400,9.000000


**Step 3: Clean and Merge ACS Data**

The ACS data needs significant wrangling to be useful. The columns have cryptic names, and the first row is a human-readable description, not data. We need to extract the relevant columns, rename them, and create a common key for merging.

- **Select and Rename Columns:** We'll identify the columns corresponding to Median Income (`DP03_0062E`), Percent Below Poverty (`S1701_C03_001E`), and Percent Uninsured (`S2701_C05_001E`) and give them meaningful names.
- **Create a Join Key:** We will create a `County_State` key (e.g., \"HOUSTON_AL\") in both the ACS and CMS dataframes to facilitate the merge. This requires cleaning and standardizing the county and state names (e.g., converting to uppercase, removing \"County\" or \"Parish\").

In [13]:
# --- Clean ACS Income Data ---
income_cleaned = acs_income[['NAME', 'DP03_0062E']].copy()
income_cleaned.columns = ['County_State_Name', 'Median Household Income']
income_cleaned = income_cleaned.iloc[1:] # Drop the descriptive header row

# --- Clean ACS Poverty Data ---
poverty_cleaned = acs_poverty[['NAME', 'S1701_C03_001E']].copy()
poverty_cleaned.columns = ['County_State_Name', 'Poverty Rate']
poverty_cleaned = poverty_cleaned.iloc[1:]

# --- Clean ACS Insurance Data ---
insurance_cleaned = acs_insurance[['NAME', 'S2701_C05_001E']].copy()
insurance_cleaned.columns = ['County_State_Name', 'Uninsured Rate']
insurance_cleaned = insurance_cleaned.iloc[1:]

# --- Merge ACS Datasets ---
acs_merged = pd.merge(income_cleaned, poverty_cleaned, on='County_State_Name', how='inner')
acs_merged = pd.merge(acs_merged, insurance_cleaned, on='County_State_Name', how='inner')

# Convert data to numeric
for col in ['Median Household Income', 'Poverty Rate', 'Uninsured Rate']:
    acs_merged[col] = pd.to_numeric(acs_merged[col], errors='coerce')

# Create Join Key
acs_merged['County'] = acs_merged['County_State_Name'].apply(lambda x: x.split(',')[0].replace(' County', '').replace(' Parish', '').upper())
acs_merged['State_Name'] = acs_merged['County_State_Name'].apply(lambda x: x.split(',')[1].strip())

# A dictionary to map full state names to abbreviations is needed for the final merge
state_map = { 'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY', 'District of Columbia': 'DC', 'Puerto Rico': 'PR'}
acs_merged['State'] = acs_merged['State_Name'].map(state_map)
acs_merged['Join_Key'] = acs_merged['County'] + '_' + acs_merged['State']

print("Merged ACS Data Shape:", acs_merged.shape)
acs_merged.head()

Merged ACS Data Shape: (854, 8)


,County_State_Name,Median Household Income,Poverty Rate,Uninsured Rate,County,State_Name,State,Join_Key
0,"Baldwin County, Alabama",72915,9.7,6.8,BALDWIN,Alabama,AL,BALDWIN_AL
1,"Calhoun County, Alabama",50780,21.3,8.9,CALHOUN,Alabama,AL,CALHOUN_AL
2,"Cullman County, Alabama",58923,18.3,10.6,CULLMAN,Alabama,AL,CULLMAN_AL
3,"DeKalb County, Alabama",43509,24.7,13.9,DEKALB,Alabama,AL,DEKALB_AL
4,"Elmore County, Alabama",72478,12.9,7.6,ELMORE,Alabama,AL,ELMORE_AL


**Step 4: Final Merge**

Finally, we merge the unified CMS dataset with the unified ACS dataset using the `Join_Key`.

In [14]:
# Create Join Key in CMS data
cms_merged['County'] = cms_merged['County'].str.upper()
cms_merged['Join_Key'] = cms_merged['County'] + '_' + cms_merged['State']

# Perform the final merge
final_df = pd.merge(cms_merged, acs_merged[['Join_Key', 'Median Household Income', 'Poverty Rate', 'Uninsured Rate']], on='Join_Key', how='inner')

print("Final Analytical DataFrame Shape:", final_df.shape)
print("\nNull values check:")
print(final_df.isnull().sum())
final_df.head()

Final Analytical DataFrame Shape: (1661, 12)

Null values check:
Facility ID                         0
Facility Name                       0
County                              0
State                               0
Hospital Type                       0
Hospital Ownership                  0
Average Excess Readmission Ratio    0
Average Mortality Rate              0
Join_Key                            0
Median Household Income             0
Poverty Rate                        0
Uninsured Rate                      0
dtype: int64


,Facility ID,Facility Name,County,State,Hospital Type,Hospital Ownership,Average Excess Readmission Ratio,Average Mortality Rate,Join_Key,Median Household Income,Poverty Rate,Uninsured Rate
0,100001,SHANDS JACKSONVILLE,DUVAL,FL,Acute Care Hospitals,Voluntary non-profit - Private,1.045220,9.733333,DUVAL_FL,69436,14.6,10.5
1,100002,BETHESDA HOSPITAL EAST,PALM BEACH,FL,Acute Care Hospitals,Voluntary non-profit - Private,1.018333,11.600000,PALM BEACH_FL,84921,10.7,11.9
2,100006,ORLANDO HEALTH,ORANGE,FL,Acute Care Hospitals,Voluntary non-profit - Private,1.073800,9.183333,ORANGE_FL,79053,12.4,11.1
3,100007,ADVENTHEALTH ORLANDO,ORANGE,FL,Acute Care Hospitals,Voluntary non-profit - Private,1.165183,7.983333,ORANGE_FL,79053,12.4,11.1
4,100008,BAPTIST HOSPITAL OF MIAMI,MIAMI-DADE,FL,Acute Care Hospitals,Voluntary non-profit - Private,0.994400,9.000000,MIAMI-DADE_FL,72311,14.0,12.2


In [ ]:
Test for Xuhan